In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from comtrade import Comtrade
import seaborn as sns

In [ ]:
import os
def get_all_Paths(file_type):
    response = []
    directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\DAT_Files"
    if file_type == 2:
        directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\CFG_FILES"
    for path in os.listdir(directory):
        full_path = os.path.join(directory, path)
        if os.path.isfile(full_path):
            response.append(full_path)
    return response

In [ ]:
dat_path = get_all_Paths(1)
cgf_path = get_all_Paths(2)

In [ ]:
file_paths = []
for index, element in enumerate(dat_path):
    file_paths.append([cgf_path[index], element])

In [ ]:
file_paths[0]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def data_frames_creation(paths):
    temp=[]
    for i in range (0, len(paths)):
        rec = Comtrade()
        rec.load(paths[i][0],paths[i][1])
        samples_list=[]
        for j in range (0, len(rec.analog)):
            samples_list.append(rec.analog[j])
        abc=pd.DataFrame(samples_list)
        abc=abc.transpose()
        abc.columns=rec.analog_channel_ids
        time=pd.Series(data=rec.time,name='Time')
        abc=pd.concat([time,abc],axis=1)
        temp.append(abc)
        samples_list=[]
    return temp

In [ ]:
data_sets=data_frames_creation(file_paths)

In [ ]:
print(len(data_sets))
print(data_sets[0])

In [ ]:
#def line_plots(cols,data,i):
    #plt.figure(figsize=(15,6))
    #plt.title('Time Distribution of '+cols)
    #plt.plot(data['Time'],data[cols],color='#E9967A')
    #plt.suptitle('Data Point' + str (i))

In [ ]:
data_sets[5].columns[1:]

## Visualization

In [ ]:
def single_point_visula(cols,data):
    plt.figure(figsize=(11,2))
    plt.title('Currents Amplitude Vs Time of '+cols)
    plt.plot(data['Time'],data[cols],color='#0FE279')
    plt.xlabel('Time', fontsize=8)
    plt.ylabel('Current (amp)', fontsize=7)

In [ ]:
def box_plots(data,b):
    plt.figure(figsize=(11,3))
    plt.title('Distribution of Data Point '+ str (b))
    sns.boxplot(data=data,color='#B0FC21')
    plt.xlabel('Phases', fontsize=9)
    plt.ylabel('Current (amp)', fontsize=10)

##### Change the number in `Data_sets` to see single point visualization

In [ ]:
data_sets[0]

In [ ]:
data_sets[0].columns[1:]

In [ ]:
for cols in data_sets[0].columns[1:]:
    single_point_visula(cols,data_sets[0])

The above Figure Shows cable failure in phase `A`

In [ ]:
for cols in data_sets[89].columns[1:]:
    single_point_visula(cols,data_sets[89])

The above Figure Shows cable failure in phase `C`

In [ ]:
for cols in data_sets[0].columns[1:]:
    single_point_visula(cols,data_sets[0])

The above Figure Shows cable failure in phase `A`

In [ ]:
for cols in data_sets[340].columns[1:]:
    single_point_visula(cols,data_sets[340])

The above Figure Shows cable failure in phase `A, B and C`

In [ ]:
for b in range(len(data_sets)):
    box_plots(data_sets[b].iloc[:,1:],b)

In [ ]:
#fig,ax=plt.subplots(nrows=1,ncols=2,figsize=(15,5))
#sns.boxplot(data=temp[90].iloc[:,1:],ax=ax[0],palette='rocket')
#ax[0].set_title('Box Plots')
#ax[1].plot(temp[90]['Time'],temp[90].iloc[:,1])
#ax[1].plot(temp[90]['Time'],temp[90].iloc[:,2])
#ax[1].plot(temp[90]['Time'],temp[90].iloc[:,3])
#ax[1].plot(temp[90]['Time'],temp[90].iloc[:,4])
#ax[1].set_title('Line Plots')
#plt.savefig('Label_values.png')

In [ ]:
box_plots(data_sets[9].iloc[:,1:],9)
box_plots(data_sets[19].iloc[:,1:],19)
box_plots(data_sets[89].iloc[:,1:],89)
box_plots(data_sets[340].iloc[:,1:],340)

## Feature Engineering 

In [ ]:
data_sets[0]

In [ ]:
engin_df=pd.read_csv('eng_features.csv')
engin_df

In [ ]:
engineered_feat=engin_df.drop(['Fault ID','Data Length'],axis=1)
engineered_feat

## Feature Extraction

###### Importing Library

In [ ]:
import tsfel

In [ ]:
data_sets[10].iloc[:,1:]

### Feature Extraction process

#### `Spectral Feature Extraction`

In [ ]:
len(data_sets[0].columns)

In [ ]:
tsfel_features_spectral=[]
for j in range (0,len(data_sets)):
    for k in range (1,len(data_sets[j].columns)):
        tsfel_initialization_s = tsfel.get_features_by_domain('spectral')
        extracted = tsfel.time_series_features_extractor(tsfel_initialization_s, data_sets[j].iloc[:,k], fs=1000)
        tsfel_features_spectral.append(extracted)

In [ ]:
len(tsfel_features_spectral)

In [ ]:
343*4

In [ ]:
tsfel_features_spectral[0]

In [ ]:
for m in range (0, len(tsfel_features_spectral)):
    spectral_df=pd.concat(tsfel_features_spectral)

In [ ]:
spectral_df

###### Reset Index

In [ ]:
spectral_df=spectral_df.reset_index(drop=True)

In [ ]:
spectral_df

##### Cross-Check of Extracted Features Value
`Whether Feature Extraction Values are Correct or Not`

In [ ]:
tsfel_initialization_spectral = tsfel.get_features_by_domain('spectral')
tsfel_initialization_temporal = tsfel.get_features_by_domain('temporal')
tsfel_initialization_statistical = tsfel.get_features_by_domain('statistical')

In [ ]:
extracted_2_spectral = tsfel.time_series_features_extractor(tsfel_initialization_spectral, data_sets[1].iloc[:,1], fs=1000)

In [ ]:
extracted_2_spectral

#### Concetanation with the Featured Engineered DataFrame

In [ ]:
data_sets[22]

In [ ]:
engineered_feat

In [ ]:
spectral_eng_df=pd.concat([engineered_feat,spectral_df],axis=1)

In [ ]:
spectral_eng_df

###### Unique Values Check

In [ ]:
one_unique=[]
def one_unique_value_check(data,col):
    value=len(data[col].unique())
    if value==1:
        one_unique.append(col)
for col in spectral_eng_df.columns:
    one_unique_value_check(spectral_eng_df,col)

In [ ]:
one_unique

#### Encoding

In [ ]:
cat_cols=spectral_eng_df.select_dtypes(include=['object']).columns

In [ ]:
cat_cols

In [ ]:
import category_encoders as ce

encoder = ce.OrdinalEncoder(cols=cat_cols,handle_missing='return_nan',return_df= True)

spectral_eng_df=encoder.fit_transform(spectral_eng_df)

In [ ]:
spectral_eng_df

In [ ]:
spectral_eng_df['Phase'].unique()

In [ ]:
fea_col_names_spec=spectral_eng_df.columns
fea_col_names_spec

#### Clustering Analysis Based on PCA

##### `Spectral Normaliztion`

In [ ]:
#### Standard Scaling
from sklearn.preprocessing import StandardScaler
standard_scaler=StandardScaler()
SS_Norm=pd.DataFrame(standard_scaler.fit_transform(spectral_eng_df))
SS_Norm.columns=fea_col_names_spec

In [ ]:
SS_Norm.head()

In [ ]:
#from sklearn.preprocessing import Normalizer
#scaler_1=Normalizer()
#normalized_spectral=pd.DataFrame(scaler_1.fit_transform(final_df))
#normalized_spectral.columns=fea_col_names_spec

In [ ]:
#normalized_spectral

###### `Feature Selection Based on PCA`

In [ ]:
from sklearn.decomposition import PCA
P_C_A_s = PCA(n_components=15)
p_c_a_output_s = P_C_A_s.fit_transform(SS_Norm)
p_c_a_output_s

In [ ]:
P_C_A_s.n_components_

In [ ]:
PCA_components_s = pd.DataFrame(p_c_a_output_s)
PCA_components_s

In [ ]:
PCA_components_s.shape

###### K-Means Clustering with Elbow Method on 25 PCA  components (Spectral Domain)

In [ ]:
np.random.seed(43)
from sklearn.cluster import KMeans

kmeans_models_s = [KMeans(n_clusters=k).fit(PCA_components_s) for k in range (1, 10)]
innertia_s = [model.inertia_ for model in kmeans_models_s]

plt.plot(range(1, 10), innertia_s,'-o',color='#783654')
plt.title('Elbow method (Spectral Domian Features)')
plt.xlabel('Number of Clusters')
plt.ylabel('Model Inertia')
plt.show()
#plt.savefig('Elbow_1.png');

###### Silhouette score

Silhouette score is used to evaluate the quality of clusters created using clustering algorithms such as K-Means in terms of how well samples are clustered with other samples that are similar to each other. 

The silhouette score of 1 means that the clusters are very dense and nicely separated. The score of 0 means that clusters are overlapping. The score of less than 0 means that data belonging to clusters may be wrong/incorrect.

The silhouette plots can be used to select the most optimal value of the K (no. of cluster) in K-means clustering.


The aspects to look out for in Silhouette plots are cluster scores below the average silhouette score, wide fluctuations in the size of the clusters, and also the thickness of the silhouette plot.

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

silhoutte_scores_s = [silhouette_score(PCA_components_s, model.labels_) for model in kmeans_models_s[1:10]]
plt.plot(range(2,10), silhoutte_scores_s, "bo-",color='#857376')
plt.xticks([2, 3, 4, 5,6,7,8,9])
plt.title('Silhouette scores vs Number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhoutte score')
#plt.savefig('Silhoutte_score.png')
plt.show()

#### Silhoutte Analysis For K-Menas


In [ ]:
##`https://github.com/krishnaik06/Silhouette-clustering-/blob/master/Untitled.ipynb`

In [ ]:
from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm
range_n_clusters = [2, 3, 4, 5, 6]
for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(PCA_components_s) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labelss = clusterer.fit_predict(PCA_components_s)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(PCA_components_s, cluster_labelss)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(PCA_components_s, cluster_labelss)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labelss == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labelss.astype(float) / n_clusters)
    ax2.scatter(PCA_components_s.iloc[:, 0], PCA_components_s.iloc[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()
###### Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

###### `Note` From the above Figures it can be seen that the ideal number of clusters are 2

In [ ]:
###### I am now going to plot components with explained variance ration and will visualize the dropoff point of variance.

In [ ]:
P_C_A_s.explained_variance_ratio_

In [ ]:
features_s = range(P_C_A_s.n_components_)
plt.figure(figsize=(13,4))
plt.bar(features_s, P_C_A_s.explained_variance_ratio_, color='#30CDA0')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features_s)
plt.title('Distribution of variance vs PCA dimensions');
plt.savefig('PCA_Visualization.png')

In [ ]:
#### silhouette_score when Cluster= 2
np.random.seed(35)
from sklearn.metrics import silhouette_score

clustering_spectral = KMeans(n_clusters=2)
clustering_spectral.fit(PCA_components_s)

print('Silhoutte score of K-Mean clustering is ' + str(silhouette_score(PCA_components_s, clustering_spectral.labels_)))

In [ ]:
# #### silhouette_score when Cluster= 4
# np.random.seed(9)
# from sklearn.metrics import silhouette_score

# clustering_spectral_4_cl = KMeans(n_clusters=4)
# clustering_spectral_4_cl.fit(PCA_components_s)

# print('Silhoutte score of K-Mean clustering is ' + str(silhouette_score(PCA_components_s, clustering_spectral_4_cl.labels_)))

In [ ]:
np.random.seed(9)
spectral_labels=clustering_spectral.labels_
spectral_labels

In [ ]:
spectral_final = pd.concat([PCA_components_s, pd.DataFrame({'pca_clusters':spectral_labels})], axis=1)
spectral_final

In [ ]:
spectral_final['pca_clusters'].value_counts()

In [ ]:
##### Clustering with all Feature (All engineered Features)
#### When Cluster Number are 2

import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(spectral_final.iloc[:,0],spectral_final.iloc[:,1], hue=spectral_final['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Distribution of cluster based on Spectral Domain Features (PCA)',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`

#### TEN principal Components

In [ ]:
ten_components=PCA_components_s.iloc[:,0:10]

In [ ]:
ten_components.to_csv('Ten_com_clustering.csv',index=False)

In [ ]:
ten_components.tail()

In [ ]:
np.random.seed(43)
from sklearn.cluster import KMeans

kmeans_models_ten_com = [KMeans(n_clusters=k).fit(ten_components) for k in range (1, 10)]
innertia_ten_com = [model.inertia_ for model in kmeans_models_ten_com]

plt.plot(range(1, 10), innertia_ten_com,'-o',color='#783654')
plt.title('Elbow method')
plt.xlabel('Number of Clusters')
plt.ylabel('Model Inertia')
plt.show()
plt.savefig('Elbow_1.png');

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

silhoutte_scores_ten_com = [silhouette_score(ten_components, model.labels_) for model in kmeans_models_ten_com[1:10]]
plt.plot(range(2,10), silhoutte_scores_ten_com, "bo-",color='#857376')
plt.xticks([2, 3, 4, 5,6,7,8,9])
plt.title('Silhouette scores vs Number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhoutte score')

plt.show()
plt.savefig('Silhoutte_score.png')

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

clustering_spectral_ten_com = KMeans(n_clusters=2)
clustering_spectral_ten_com.fit(ten_components)

print('Silhoutte score of K-Mean clustering is ' + str(silhouette_score(ten_components, clustering_spectral_ten_com.labels_)))

In [ ]:
np.random.seed(9)
spectral_labels_ten_com=clustering_spectral_ten_com.labels_
spectral_labels_ten_com

In [ ]:
spectral_res_ten_com = pd.concat([ten_components, pd.DataFrame({'pca_clusters':spectral_labels_ten_com})], axis=1)
spectral_res_ten_com.sample(3)

In [ ]:
spectral_res_ten_com['pca_clusters'].value_counts()

In [ ]:
##### Clustering with all Feature (All engineered Features)
#### When Cluster Number are 2

import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(spectral_res_ten_com.iloc[:,0],spectral_res_ten_com.iloc[:,1], hue=spectral_res_ten_com['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Distribution of cluster based on Ten PC',fontsize=13)
plt.legend()
plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`

## Predicted Failure Verification

In [ ]:
#labels_df=pd.read_csv('labels.csv')
#labels_df

In [ ]:
#labels_df['fault_status'].value_counts()

In [ ]:
spectral_res_ten_com

In [ ]:
spectral_res_ten_com['pca_clusters'].value_counts()

In [ ]:
predicted_good=spectral_res_ten_com[spectral_res_ten_com['pca_clusters']==0]

In [ ]:
predicted_failure=spectral_res_ten_com[spectral_res_ten_com['pca_clusters']==1]

In [ ]:
predicted_failure

In [ ]:
len(predicted_failure)

In [ ]:
predicted_failure_index=predicted_failure.index

In [ ]:
predicted_failure_index

In [ ]:
#actual_failure=labels_df[labels_df['fault_status']=='faulty']

In [ ]:
predicted_failure

In [ ]:
#actual_failure_index=actual_failure.index

In [ ]:
#actual_failure_index

In [ ]:
#common_errors=list(set(actual_failure_index).intersection(predicted_failure_index))

In [ ]:
#len(common_errors)

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_failure.iloc[:,0],predicted_failure.iloc[:,1], hue=predicted_failure['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Only Predicted Failure cluster based on Ten PC',fontsize=13)
plt.legend(labels=['Predicted Failure Cluster Only'],)
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
#errors_sort=sorted(common_errors)
#errors_sort

In [ ]:
engin_df.head()

In [ ]:
predicted_failure_index

In [ ]:
Faulty_pred_ids=[]
for i in predicted_failure_index:
    Faulty_pred_ids.append(engin_df.loc[i])
Faulty_pred_ids=pd.DataFrame(Faulty_pred_ids)

In [ ]:
Faulty_pred_ids

In [ ]:
Faulty_pred_ids['Phase'].unique()

In [ ]:
import os
def getPath(file_id, file_type):
    response = ''
    directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\DAT_Files"
    if file_type == 2:
        directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\CFG_FILES"
    for path in os.listdir(directory):
        full_path = os.path.join(directory, path)
        if os.path.isfile(full_path):
            if str(file_id) in full_path:
                response = full_path
    return response

In [ ]:
failure_paths=[]
for index, row in Faulty_pred_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    failure_paths.append([CFG_Path, dAT_path])

In [ ]:
len(failure_paths)

In [ ]:
faulty_phases=[]
for index, row in Faulty_pred_ids.iterrows():
    faulty_phases.append(row['Phase'])

In [ ]:
data_sets_fal=data_frames_creation(failure_paths)

In [ ]:
len(data_sets_fal)

In [ ]:
col_names=[ 'MPI3p1:I A', 'MPI3p1:I B', 'MPI3p1:I C', 'MPI3p1:IN']

In [ ]:
def pred_fal_good_Visualization(dataframe, wrong_phases,columns):
    for f in range (0,len(dataframe)):
        if wrong_phases[f]=='A':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[0])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I A'],color='#E9967A')
        if wrong_phases[f]=='B':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[1])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I B'],color='#108A0D')
        if wrong_phases[f]=='C':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[2])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I C'],color='#0282B9')
        if wrong_phases[f]=='N':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[3])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:IN'],color='#C794F2')

In [ ]:
pred_fal_good_Visualization(data_sets_fal,faulty_phases,col_names)

In [ ]:
predicted_failure_reset=predicted_failure.reset_index(drop=True)

In [ ]:
false_neg=Faulty_pred_ids.iloc[[13,15]]
false_neg

In [ ]:
not_failure_paths=[]
for index, row in false_neg.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    not_failure_paths.append([CFG_Path, dAT_path])

In [ ]:
not_failure_paths

In [ ]:
not_faulty_phases=[]
for index, row in false_neg.iterrows():
    not_faulty_phases.append(row['Phase'])

In [ ]:
not_faulty_phases

In [ ]:
false_neg_data_sets=data_frames_creation(not_failure_paths)

In [ ]:
pred_fal_good_Visualization(false_neg_data_sets,not_faulty_phases,col_names)

In [ ]:
predicted_failure_reset.head()

In [ ]:
fal_drop=predicted_failure_reset.iloc[[13,15]]

In [ ]:
fal_drop

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_failure.iloc[:,0],predicted_failure.iloc[:,1], palette='viridis', s=70, alpha=0.8).set_title('Predicted Failure cluster only based on Ten PC',fontsize=13)
sns.scatterplot(fal_drop.iloc[:,0],fal_drop.iloc[:,1],  s=70, alpha=0.8, palette='rocket')
#plt.legend()
plt.legend(labels=["True Failure","False Failure"])
plt.savefig('failure_only')
plt.show()

###### `********************************************************************************************************************`

## Predicted Good Verification

In [ ]:
predicted_good.head()

In [ ]:
predicted_good_index=predicted_good.index
predicted_good_index

In [ ]:
len(predicted_good_index)

In [ ]:
#actual_positive=labels_df[labels_df['fault_status']=='good condition']

In [ ]:
#actual_positive.head()

In [ ]:
#actual_positive_index=actual_positive.index

In [ ]:
#len(actual_positive_index)

In [ ]:
#common_positive=list(set(actual_positive_index).intersection(predicted_good_index))

In [ ]:
#common_positive=sorted(common_positive)

In [ ]:
engin_df

In [ ]:
Good_pred_ids=[]
for i in predicted_good_index:
    Good_pred_ids.append(engin_df.loc[i])
Good_pred_ids=pd.DataFrame(Good_pred_ids)

In [ ]:
Good_pred_ids.head()

In [ ]:
pred_good_paths=[]
for index, row in Good_pred_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    pred_good_paths.append([CFG_Path, dAT_path])

In [ ]:
pred_good_paths[0]

In [ ]:
pred_good_phases=[]
for index, row in Good_pred_ids.iterrows():
    pred_good_phases.append(row['Phase'])

In [ ]:
pred_good_data_sets=data_frames_creation(pred_good_paths)

In [ ]:
len(pred_good_data_sets)

In [ ]:
pred_fal_good_Visualization(pred_good_data_sets,pred_good_phases,col_names)

In [ ]:
false_pred_good=[0,22,26,30,37,41,45,49,53,55,60,64,90,91,96,101,106,107,112,118,156,160,164,168,172,176,179,182,185,211,232,236,240,244,248,252,256,257,262,267,272,276,280,284,
288,292,310,324,328,329,334,339,341,351,359,367,371,375,379,386,400,404,408,412,413,417,422,427,432,436,440,444,448,470,475,480,485,501,505,509,513,517,522,527,532,536,540,544,545,567,572,577,582,586,590,594,595,620,625,629,634,639,643,647,651,652,671,689,691,696,701,706,710,714,718,719,744,749,754,759,763,767,792,796,801,806,811,815,819,823,824,846,851,856,861,865,893,897,902,907,912,916,920,924,946,951,956,961,965,984,989,994,999,1003,1007,1008,1030,1035,1040,1045,1049,1050,1072,1077,1082,1087,1091,1095,1099,1103,1122,1123,1124,1126,1127,1128,1130,1131,1132,1134,1135,1136,1138,1139,1140,1142,1143,1144,1146,1147,1148,1150,1151,1152,1154,1155,1156,1158,1159,1160,1162,1163,1164,1166,1167,1168,1170,1171,1172,1174,1175,1176,1178,1179,1180,1182,1183,1184,1186,1187,1188,1190,1191,1192,1194,1195,1196,1198,1199,1200,1202,1203,1204,1206,1207,1208,1210,1211,1212,1214,1215,1216,1218,1219,1220
]

In [ ]:
len(false_pred_good)

In [ ]:
predicted_good_reset=predicted_good.reset_index(drop=True)

In [ ]:
predicted_good_reset.head()

In [ ]:
false_pred_good_df=predicted_good_reset.iloc[false_pred_good]

In [ ]:
false_pred_good_df

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_good.iloc[:,0],predicted_good.iloc[:,1], hue=predicted_good['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title(' Only Predicted Good cluster based on Ten PC',fontsize=13)
plt.legend(labels=['Predicted Good Cluster Only'],loc='upper left')
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_good.iloc[:,0],predicted_good.iloc[:,1], palette='viridis', s=70, alpha=0.8).set_title(' Predicted Good cluster only based on Ten PC',fontsize=13)
sns.scatterplot(false_pred_good_df.iloc[:,0],false_pred_good_df.iloc[:,1],  s=70, alpha=0.8, palette='rocket')
#plt.legend()
plt.legend(labels=["True Positive","False Positive"])
plt.savefig('good_pred')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(spectral_res_ten_com.iloc[:,0],spectral_res_ten_com.iloc[:,1], hue=spectral_res_ten_com['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Distribution of cluster based on Ten PC',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`

### Experiment

In [ ]:
temp_experiment=[]
for i in false_pred_good: 
    #print(i)
    temp_experiment.append(pred_good_data_sets[i])

In [ ]:
len(temp_experiment)

In [ ]:
phase_experiment=[]
for i in false_pred_good:
    phase_experiment.append(pred_good_phases[i])

In [ ]:
phase_experiment

In [ ]:
pred_fal_good_Visualization(temp_experiment,phase_experiment,col_names)

###### `********************************************************************************************************************`

## Gaussian Mixture Model

In [ ]:
ten_components.head()

In [ ]:
#n_components = np.arange(1, 10)
#models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(ten_components)
          #for n in n_components]

#plt.plot(n_components, [m.bic(ten_components) for m in models], label='BIC')
#plt.plot(n_components, [m.aic(ten_components) for m in models], label='AIC')
#plt.legend(loc='best')
#plt.xlabel('n_components');

In [ ]:
np.random.seed(40)
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=2).fit(ten_components)

In [ ]:
np.random.seed(40)
gmm_labels=gmm.predict(ten_components)

In [ ]:
gmm_df = pd.concat([ten_components, pd.DataFrame({'pca_clusters':gmm_labels})], axis=1)

In [ ]:
print('Silhoutte score of GMM clustering is ' + str(silhouette_score(ten_components, gmm_labels)))

In [ ]:
gmm_df

In [ ]:
gmm_df['pca_clusters'].value_counts()

In [ ]:
probs = gmm.predict_proba(ten_components)

In [ ]:
##### Clustering with all Feature (All engineered Features)
#### When Cluster Number are 2

import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(gmm_df.iloc[:,0],gmm_df.iloc[:,1], hue=gmm_df['pca_clusters'], palette='magma', s=70, alpha=0.8).set_title('Distribution of cluster Based on Gaussian Mixture Model',fontsize=13)
plt.xlabel('First PCA')
plt.ylabel('2nd PCA')
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

## Predicted Failure Verification by GMM

In [ ]:
predicted_failure_gmm=gmm_df[gmm_df['pca_clusters']==1]

In [ ]:
predicted_failure_gmm

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_failure_gmm.iloc[:,0],predicted_failure_gmm.iloc[:,1],color='#07EA30' , s=70, alpha=0.8).set_title('Only Predicted Failure cluster by GMM',fontsize=13)
plt.legend(labels=['Predicted Failure Cluster Only'])
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
predicted_failure_gmm_index=predicted_failure_gmm.index

In [ ]:
engin_df.head()

In [ ]:
Faulty_pred_gmm_ids=[]
for i in predicted_failure_gmm_index:
    Faulty_pred_gmm_ids.append(engin_df.loc[i])
Faulty_pred_gmm_ids=pd.DataFrame(Faulty_pred_gmm_ids)

In [ ]:
Faulty_pred_gmm_ids

In [ ]:
gmm_failure_paths=[]
for index, row in Faulty_pred_gmm_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_failure_paths.append([CFG_Path, dAT_path])

In [ ]:
gmm_faulty_phases=[]
for index, row in Faulty_pred_gmm_ids.iterrows():
    gmm_faulty_phases.append(row['Phase'])

In [ ]:
print(len(gmm_failure_paths))
print(len(gmm_faulty_phases))

In [ ]:
gmm_pred_fal_data=data_frames_creation(gmm_failure_paths)

In [ ]:
len(gmm_pred_fal_data)

In [ ]:
pred_fal_good_Visualization(gmm_pred_fal_data,gmm_faulty_phases,col_names)

`Note`

    The Above Visulization Proves that all the point that was predicted Failure by the GMM is actually Failure except two

In [ ]:
predicted_failure_gmm.head(4)

In [ ]:
predicted_failure_gmm_reset=predicted_failure_gmm.reset_index(drop=True)

In [ ]:
fal_drop_gmm=predicted_failure_gmm_reset.iloc[[25,27]]

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_failure_gmm.iloc[:,0],predicted_failure_gmm.iloc[:,1], color='#677666', s=70, alpha=0.4).set_title('Predicted Failure cluster only based on GMM',fontsize=13)
sns.scatterplot(fal_drop_gmm.iloc[:,0],fal_drop_gmm.iloc[:,1],  s=70, alpha=0.8, color='#F8652A')
#plt.legend()
plt.legend(labels=["True Failure","False Failure"])
#plt.savefig('failure_only')
plt.show()

###### `********************************************************************************************************************`

### Hyperparameters Tunning (Only Spectral Domian)

In [ ]:
np.random.seed(40)
from sklearn.mixture import GaussianMixture
gmm_diag = GaussianMixture(n_components=2,covariance_type='diag').fit(ten_components)

In [ ]:
gmm_diag.get_params()

In [ ]:
np.random.seed(40)
gmm_dia_labels=gmm_diag.predict(ten_components)

In [ ]:
gmm_dia_df = pd.concat([ten_components, pd.DataFrame({'pca_clusters':gmm_dia_labels})], axis=1)

In [ ]:
print('Silhoutte score of GMM clustering is ' + str(silhouette_score(ten_components, gmm_dia_labels)))

In [ ]:
gmm_dia_df.head()

In [ ]:
gmm_dia_df

In [ ]:
gmm_dia_df['pca_clusters'].value_counts()

In [ ]:
##### Clustering with all Feature (All engineered Features)
#### When Cluster Number are 2

import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(gmm_dia_df.iloc[:,0],gmm_dia_df.iloc[:,1], hue=gmm_dia_df['pca_clusters'], palette='magma', s=70, alpha=0.8).set_title('Distribution of cluster Based on Gaussian Mixture Model (Diagonal)',fontsize=13)
plt.xlabel('First PCA')
plt.ylabel('2nd PCA')
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`

## Predicted Failure Verification by GMM (Co-Variance type: Diagonal)

In [ ]:
pred_fal_gmm_dia=gmm_dia_df[gmm_dia_df['pca_clusters']==1]

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia.iloc[:,0],pred_fal_gmm_dia.iloc[:,1],color='#08BF4B' , s=70, alpha=0.8).set_title('Only Predicted Failure cluster by GMM',fontsize=13)
plt.legend(labels=['Predicted Failure Cluster Only'])
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
predicted_failure_gmm_index=pred_fal_gmm_dia.index

In [ ]:
engin_df.head()

In [ ]:
Faulty_pred_gmm_ids=[]
for i in predicted_failure_gmm_index:
    Faulty_pred_gmm_ids.append(engin_df.loc[i])
Faulty_pred_gmm_ids=pd.DataFrame(Faulty_pred_gmm_ids)

In [ ]:
len(Faulty_pred_gmm_ids)

In [ ]:
gmm_dia_failure_paths=[]
for index, row in Faulty_pred_gmm_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_dia_failure_paths.append([CFG_Path, dAT_path])

In [ ]:
gmm_dia_faulty_phases=[]
for index, row in Faulty_pred_gmm_ids.iterrows():
    gmm_dia_faulty_phases.append(row['Phase'])

In [ ]:
print(len(gmm_dia_failure_paths))
print(len(gmm_dia_faulty_phases))

In [ ]:
gmm_dia_fal_data=data_frames_creation(gmm_dia_failure_paths)

In [ ]:
len(gmm_dia_fal_data)

In [ ]:
pred_fal_good_Visualization(gmm_dia_fal_data,gmm_dia_faulty_phases,col_names)

In [ ]:
false_failure_diag=[2,32,34,52,60,78,134,153,275,293,320]

In [ ]:
len(false_failure_diag)

In [ ]:
len(pred_fal_gmm_dia)

In [ ]:
predicted_failure_gmm_dia_reset=pred_fal_gmm_dia.reset_index(drop=True)

In [ ]:
predicted_failure_gmm_dia_reset

In [ ]:
fal_drop_gmm_dia=pred_fal_gmm_dia.iloc[false_failure_diag]

In [ ]:
len(fal_drop_gmm_dia)

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia.iloc[:,0],pred_fal_gmm_dia.iloc[:,1], color='#677666', s=70, alpha=0.4).set_title('Predicted Failure cluster only based on GMM (Co-Variance type: Diagonal)',fontsize=13)
sns.scatterplot(fal_drop_gmm_dia.iloc[:,0],fal_drop_gmm_dia.iloc[:,1],  s=70, alpha=0.8, color='#F8652A')
#plt.legend()
plt.legend(labels=["True Failure","False Failure"])
#plt.savefig('failure_only')
plt.show()

## Predicted Good Verification

In [ ]:
predicted_good_gmm_dia=gmm_dia_df[gmm_dia_df['pca_clusters']==0]

In [ ]:
predicted_good_gmm_dia.head()

In [ ]:
Gmm_dia_predicted_good_index=predicted_good_gmm_dia.index
Gmm_dia_predicted_good_index

In [ ]:
len(Gmm_dia_predicted_good_index)

In [ ]:
Good_pred_ids_gmm_dia=[]
for i in Gmm_dia_predicted_good_index:
    Good_pred_ids_gmm_dia.append(engin_df.loc[i])
Good_pred_ids_gmm_dia=pd.DataFrame(Good_pred_ids_gmm_dia)

In [ ]:
Good_pred_ids_gmm_dia.head()

In [ ]:
gmm_dia_paths_g=[]
for index, row in Good_pred_ids_gmm_dia.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_dia_paths_g.append([CFG_Path, dAT_path])

In [ ]:
gmm_dia_g_phases=[]
for index, row in Good_pred_ids_gmm_dia.iterrows():
    gmm_dia_g_phases.append(row['Phase'])

In [ ]:
len(gmm_dia_g_phases)

In [ ]:
pred_good_gmm_dia=data_frames_creation(gmm_dia_paths_g)

In [ ]:
len(pred_good_gmm_dia)

In [ ]:
pred_fal_good_Visualization(pred_good_gmm_dia,gmm_dia_g_phases,col_names)

In [ ]:
predicted_good_gmm_dia.head()

In [ ]:
gmm_dia_false_g=[93,99]

In [ ]:
predicted_good_gmm_dia_index_resset=predicted_good_gmm_dia.reset_index(drop=True)

In [ ]:
good_pred_false_df=predicted_good_gmm_dia_index_resset.iloc[gmm_dia_false_g]

In [ ]:
good_pred_false_df

In [ ]:
len(predicted_good_gmm_dia)

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_good_gmm_dia.iloc[:,0],predicted_good_gmm_dia.iloc[:,1], color='#677666',  s=70, alpha=0.8).set_title(' Only Predicted Good cluster based GMM (Diagonal)',fontsize=13)
plt.legend(labels=['Predicted Good Cluster Only'],loc='upper left')
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_good_gmm_dia.iloc[:,0],predicted_good_gmm_dia.iloc[:,1], color='#677666', s=70, alpha=0.8).set_title(' True Positive Vs False Positive Baes on GMM (Diagonal)',fontsize=13)
sns.scatterplot(good_pred_false_df.iloc[:,0],good_pred_false_df.iloc[:,1], color='#F8652A', s=70, alpha=0.8)
#plt.legend()
plt.legend(labels=["True Positive","False Positive"])
#plt.savefig('good_pred')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (20,7))
sns.scatterplot(pred_fal_gmm_dia.iloc[:,0],pred_fal_gmm_dia.iloc[:,1], color='#677666', s=70, alpha=0.6).set_title('Predicted Failure cluster only based on GMM',fontsize=13)
sns.scatterplot(fal_drop_gmm.iloc[:,0],fal_drop_gmm.iloc[:,1],  s=70, alpha=0.8, color='#F8652A')

sns.scatterplot(predicted_good_gmm_dia.iloc[:,0],predicted_good_gmm_dia.iloc[:,1], color='#7926FA', s=70, alpha=0.8).set_title(' Gaussian Mixture Model (Diagonal)',fontsize=13)
sns.scatterplot(good_pred_false_df.iloc[:,0],good_pred_false_df.iloc[:,1], color='#26FAA0', s=70, alpha=0.8)
plt.legend(labels=["True Failure","False Failure","True Positive","False Positive"])

#plt.xlim(-5, )
#plt.ylim(-40, 40)
#plt.savefig('failure_only')

plt.show()

###### `********************************************************************************************************************`

#### DBSCAN Results

In [ ]:
pred_fal_or_outlier=396
false_outliers=10
pred_good_dbscan=976
incor_good_preds_dbscan=8

In [ ]:
Silhouette_score_Db_scan=0.5409140073831038
calinski_harabasz_score_dbscan=514.6113749541014
davies_bouldin_score_dbscan=1.3294301379685611

## Models Evaluation

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

#### Silhouette score Cluster Wise

##### By Hand

In [ ]:
exp_1=pd.Series(sample_silhouette_values,name='Sample_Wise_SS')

In [ ]:
Dia_wi_SSS=pd.concat([gmm_dia_df,exp_1],axis=1)

In [ ]:
Dia_wi_SSS

In [ ]:
Dia_wi_SSS['Sample_Wise_SS'][Dia_wi_SSS['pca_clusters']==0].mean()

In [ ]:
Dia_wi_SSS['Sample_Wise_SS'][Dia_wi_SSS['pca_clusters']==1].mean()

###### From Sklearn Metrics

In [ ]:
from sklearn.metrics import silhouette_samples
sample_silhouette_values = silhouette_samples(ten_components, gmm_dia_labels)

In [ ]:
len(sample_silhouette_values)

In [ ]:
means_lst = []
for label in range(2):
    means_lst.append(sample_silhouette_values[gmm_dia_labels == label].mean())

In [ ]:
means_lst

`****************************************************`

In [ ]:
np.random.seed(9)
K_Means_2=KMeans(n_clusters=2)
K_Means_2.fit(ten_components)
K_mean_labels_2=K_Means_2.labels_
##########################
##########################
# K_Means_4=KMeans(n_clusters=4)
# K_Means_4.fit(ten_components)
# K_mean_labels_4=K_Means_4.labels_
##########################
##########################
gmm_spher_2 = GaussianMixture(n_components=2,covariance_type='spherical').fit(ten_components)
gmm_spheri_labels_2=gmm_spher_2.predict(ten_components)
##########################
##########################
# gmm_spher_4 = GaussianMixture(n_components=4,covariance_type='spherical').fit(ten_components)
# gmm_spheri_labels_4=gmm_spher_4.predict(ten_components)
##########################
##########################
gmm_full_2 = GaussianMixture(n_components=2,covariance_type='full').fit(ten_components)
gmm_full_labels_2=gmm_full_2.predict(ten_components)
##########################
##########################
# gmm_full_4 = GaussianMixture(n_components=4,covariance_type='full').fit(ten_components)
# gmm_full_labels_4=gmm_full_4.predict(ten_components)
##########################
##########################
gmm_dia_2 = GaussianMixture(n_components=2,covariance_type='diag').fit(ten_components)
gmm_dia_labels_2=gmm_dia_2.predict(ten_components)
##########################
##########################
# gmm_dia_4 = GaussianMixture(n_components=4,covariance_type='diag').fit(ten_components)
# gmm_dia_labels_4=gmm_dia_4.predict(ten_components)

#### Silhouette score

In [ ]:
Silhouette_score_K_means_2_clus=silhouette_score(ten_components,spectral_labels_ten_com)
#Silhouette_score_K_means_4_clus=silhouette_score(ten_components,K_mean_labels_4.labels_)
Silhouette_score_Gmm_sphe_2=silhouette_score(ten_components,gmm_spheri_labels_2)
#Silhouette_score_Gmm_sphe_4=silhouette_score(ten_components,gmm_spheri_labels_4)
Silhouette_score_Gmm_full_2=silhouette_score(ten_components,gmm_full_labels_2)
#Silhouette_score_Gmm_full_4=silhouette_score(ten_components,gmm_full_labels_4)
Silhouette_score_Gmm_dia_2=silhouette_score(ten_components,gmm_dia_labels_2)
#Silhouette_score_Gmm_dia_4=silhouette_score(ten_components,gmm_dia_labels_4)

In [ ]:
print('For n-cluster=2')
print('**************************')
print('Silhouette score for K-Means ',Silhouette_score_K_means_2_clus)
print('Silhouette score for Gmm Covaraince type (Full) ',Silhouette_score_Gmm_full_2)
print('Silhouette score for Gmm Covaraince type (Diagonal) ',Silhouette_score_Gmm_dia_2)
print('Silhouette score for Gmm Covaraince type (Spherical) ',Silhouette_score_Gmm_sphe_2)
print('          ')
print('**************************')
#print('For n-cluster=4')
#print('Silhouette score for K-Means ',Silhouette_score_K_means_4_clus)
#print('Silhouette score for Gmm Covaraince type (Full) ',Silhouette_score_Gmm_full_4)
#print('Silhouette score for Gmm Covaraince type (Diagonal) ',Silhouette_score_Gmm_dia_4)
#print('Silhouette score for Gmm Covaraince type (Spherical) ',Silhouette_score_Gmm_sphe_4)

#### Davis Bouldin Index

In [ ]:
Davis_bouldin_K_means_2_clus=davies_bouldin_score(ten_components,spectral_labels_ten_com)
#Davis_bouldin_K_means_4_clus=davies_bouldin_score(ten_components,clustering_spectral_4_cl.labels_)
Davis_bouldin_Gmm_sphe_2=davies_bouldin_score(ten_components,gmm_spheri_labels_2)
#Davis_bouldin_Gmm_sphe_4=davies_bouldin_score(ten_components,gmm_spheri_labels_4)
Davis_bouldin_Gmm_full_2=davies_bouldin_score(ten_components,gmm_full_labels_2)
#Davis_bouldin_Gmm_full_4=davies_bouldin_score(ten_components,gmm_full_labels_4)
Davis_bouldin_Gmm_dia_2=davies_bouldin_score(ten_components,gmm_dia_labels_2)
#Davis_bouldin_Gmm_dia_4=davies_bouldin_score(ten_components,gmm_dia_labels_4)

In [ ]:
print('For n-cluster=2')
print('**************************')
print('Davis Bouldin Index with K-Means ',Davis_bouldin_K_means_2_clus)
print('Davis Bouldin Index with Gmm Covaraince type (Full) ',Davis_bouldin_Gmm_full_2)
print('Davis Bouldin Index with Gmm Covaraince type (Diagonal) ',Davis_bouldin_Gmm_dia_2)
print('Davis Bouldin Index with Gmm Covaraince type (Spherical) ',Davis_bouldin_Gmm_sphe_2)
print('          ')
print('**************************')
#print('For n-cluster=4')
#print('Davis Bouldin Index with K-Means ',Davis_bouldin_K_means_4_clus)
#print('Davis Bouldin Index with Gmm Covaraince type (Full) ',Davis_bouldin_Gmm_full_4)
#print('Davis Bouldin Index with Gmm Covaraince type (Diagonal) ',Davis_bouldin_Gmm_dia_4)
#print('Davis Bouldin Index with Gmm Covaraince type (Spherical) ',Davis_bouldin_Gmm_sphe_4)

##### calinski_harabasz_score

In [ ]:
calinski_harabasz_K_means_2_clus=calinski_harabasz_score(ten_components,spectral_labels_ten_com)
#calinski_harabasz_K_means_4_clus=calinski_harabasz_score(ten_components,clustering_spectral_4_cl.labels_)
calinski_harabasz_gmm_sph_2=calinski_harabasz_score(ten_components,gmm_spheri_labels_2)
#calinski_harabasz_gmm_sph_4=calinski_harabasz_score(ten_components,gmm_spheri_labels_4)
calinski_harabasz_gmm_full_2=calinski_harabasz_score(ten_components,gmm_full_labels_2)
#calinski_harabasz_gmm_full_4=calinski_harabasz_score(ten_components,gmm_full_labels_4)
calinski_harabasz_gmm_dia_2=calinski_harabasz_score(ten_components,gmm_dia_labels_2)
#calinski_harabasz_gmm_dia__4=calinski_harabasz_score(ten_components,gmm_dia_labels_4)

In [ ]:
print('For n-cluster=2')
print('**************************')
print('Calinski Harabasz Score with K-Means ',calinski_harabasz_K_means_2_clus)
print('Calinski Harabasz Score with Gmm Covaraince type (Full) ',calinski_harabasz_gmm_full_2)
print('Calinski Harabasz Score with Gmm Covaraince type (Diagonal) ',calinski_harabasz_gmm_dia_2)
print('Calinski Harabasz Score with Gmm Covaraince type (Spherical) ',calinski_harabasz_gmm_sph_2)
print('          ')
print('**************************')
#print('For n-cluster=4')
#print('**************************')
#print('Calinski Harabasz Score with K-Means ',calinski_harabasz_K_means_4_clus)
#print('Calinski Harabasz Score with Gmm Covaraince type (Full) ',calinski_harabasz_gmm_full_4)
#print('Calinski Harabasz Score with Gmm Covaraince type (Diagonal) ',calinski_harabasz_gmm_dia__4)
#print('Calinski Harabasz Score with Gmm Covaraince type (Spherical) ',calinski_harabasz_gmm_sph_4)

#### Evaluation Metrics DataFrame

In [ ]:
Evaluation_df=pd.DataFrame({
    'Silhouette Score':[Silhouette_score_K_means_2_clus,Silhouette_score_Gmm_full_2,Silhouette_score_Gmm_dia_2,Silhouette_score_Gmm_sphe_2,Silhouette_score_Db_scan],
    'Davis Bouldin Index': [Davis_bouldin_K_means_2_clus,Davis_bouldin_Gmm_full_2,Davis_bouldin_Gmm_dia_2,Davis_bouldin_Gmm_sphe_2,davies_bouldin_score_dbscan],
    'Calinski Harabasz Score':[calinski_harabasz_K_means_2_clus,calinski_harabasz_gmm_full_2,calinski_harabasz_gmm_dia_2,calinski_harabasz_gmm_sph_2,calinski_harabasz_score_dbscan]
},
    index=['K-Means','GMM (Full)','GMM (Diagonal)','GMM (Spherical)','DBSCAN'])

In [ ]:
Evaluation_df

###### `********************************************************************************************************************`

#### Without PCA And Correlation

In [ ]:
SS_Norm.sample(3)

In [ ]:
np.random.seed(43)
from sklearn.cluster import KMeans

kmeans_models_without_PCA_Co = [KMeans(n_clusters=k).fit(SS_Norm) for k in range (1, 10)]
innertia_without = [model.inertia_ for model in kmeans_models_without_PCA_Co]

plt.plot(range(1, 10), innertia_without,'-o',color='#783654')
plt.title('Elbow method')
plt.xlabel('Number of Clusters')
plt.ylabel('Model Inertia')
plt.show()
#plt.savefig('Elbow_1.png');

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

silhoutte_scores_without = [silhouette_score(SS_Norm, model.labels_) for model in kmeans_models_without_PCA_Co[1:10]]
plt.plot(range(2,10), silhoutte_scores_without, "bo-",color='#857376')
plt.xticks([2, 3, 4, 5,6,7,8,9])
plt.title('Silhouette scores vs Number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhoutte score')
#plt.savefig('Silhoutte_score.png')
plt.show()

`NOTE` No Change in clustering results 

###### `********************************************************************************************************************`

#### Clustering Analysis Based on Correlation

##### `Feature Selection Based on Correlation`

In [ ]:
final_df.head(4)

In [ ]:
for_corr=final_df

In [ ]:
for_corr.sample(3)

In [ ]:
corr_matrix=np.triu(for_corr.corr())
corr_matrix

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(for_corr, 0.8)
len(set(corr_features))

In [ ]:
corr_features=list(corr_features)
corr_features

In [ ]:
correlated_removed=for_corr.drop(corr_features,axis=1)

In [ ]:
correlated_removed.head()

###### `Normalization`

In [ ]:
correlated_removed_cols=correlated_removed.columns
correlated_removed_cols

In [ ]:
#### Standard Scaling
from sklearn.preprocessing import StandardScaler
standard_scaler_corr=StandardScaler()
SS_Norm_for_corr=pd.DataFrame(standard_scaler_corr.fit_transform(correlated_removed))
SS_Norm_for_corr.columns=correlated_removed_cols

In [ ]:
#from sklearn.preprocessing import Normalizer
#scaler_2=Normalizer()
#normalized_spectral_2=pd.DataFrame(scaler_2.fit_transform(correlated_removed))
#normalized_spectral_2.columns=correlated_removed_cols

In [ ]:
SS_Norm_for_corr.head()

In [ ]:
np.random.seed(40)
from sklearn.cluster import KMeans

kmeans_models_s_corr = [KMeans(n_clusters=k).fit(SS_Norm_for_corr) for k in range (1, 10)]
innertia_s_corr = [model.inertia_ for model in kmeans_models_s_corr]

plt.plot(range(1, 10), innertia_s_corr,'-o',color='#546546')
plt.title('Elbow method')
plt.xlabel('Number of Clusters')
plt.ylabel('Model Inertia')
plt.show()
#plt.savefig('Elbow_1.png');

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

silhoutte_scores_s_2 = [silhouette_score(SS_Norm_for_corr, model.labels_) for model in kmeans_models_s_corr[1:10]]
plt.plot(range(2,10), silhoutte_scores_s_2, "bo-",color='#857376')
plt.xticks([2, 3, 4, 5,6,7,8,9])
plt.title('Silhouette scores vs Number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhoutte score')
#plt.savefig('Silhoutte_score.png')
plt.show()

In [ ]:
np.random.seed(100)
from sklearn.metrics import silhouette_score

clustering_spectral_2 = KMeans(n_clusters=2)
clustering_spectral_2.fit(SS_Norm_for_corr)

print('Silhoutte score of K-Mean clustering is ' + str(silhouette_score(SS_Norm_for_corr, clustering_spectral_2.labels_)))

In [ ]:
spectral_labels_2=clustering_spectral_2.labels_
spectral_labels_2

In [ ]:
spectral_final_2 = pd.concat([SS_Norm_for_corr, pd.DataFrame({'pca_clusters':spectral_labels_2})], axis=1)
spectral_final_2.head()

In [ ]:
spectral_final_2['pca_clusters'].value_counts()

In [ ]:
##### Clustering with Features Based on Correlation
#### When Cluster Number are 4
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(spectral_final_2.iloc[:,5],spectral_final_2.iloc[:,7], hue=spectral_final_2['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Distribution of cluster based on Spectral Domain Features (correlation Based)',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

`Note` Poor Results based on correlation Method

###### `********************************************************************************************************************`

#### Only Spectral Features

In [ ]:
spectral_df.sample(4)

In [ ]:
spectral_df_3=spectral_df.columns
spectral_df_3

In [ ]:
#### Normalization
from sklearn.preprocessing import Normalizer
scaler_3=Normalizer()
normalized_spectral_3=pd.DataFrame(scaler_3.fit_transform(spectral_df))
normalized_spectral_3.columns=spectral_df_3

In [ ]:
#### PCA
from sklearn.decomposition import PCA
P_C_A_3 = PCA(n_components=25)
p_c_a_output_s_3 = P_C_A_3.fit_transform(normalized_spectral_3)
p_c_a_output_s_3

In [ ]:
PCA_components_s_3 = pd.DataFrame(p_c_a_output_s_3)

In [ ]:
PCA_components_s_3.sample(4)

In [ ]:
##### K-Means
np.random.seed(40)
from sklearn.cluster import KMeans

kmeans_models_s_3 = [KMeans(n_clusters=k).fit(PCA_components_s_3) for k in range (1, 10)]
innertia_s_3 = [model.inertia_ for model in kmeans_models_s_3]

plt.plot(range(1, 10), innertia_s_3,'-o',color='#783654')
plt.title('Elbow method')
plt.xlabel('Number of Clusters')
plt.ylabel('Model Inertia')
plt.show()
#plt.savefig('Elbow_1.png');

In [ ]:
##### silhouette_score
np.random.seed(40)
from sklearn.metrics import silhouette_score

silhoutte_scores_s_3 = [silhouette_score(PCA_components_s_3, model.labels_) for model in kmeans_models_s_3[1:10]]
plt.plot(range(2,10), silhoutte_scores_s_3, "bo-",color='#857376')
plt.xticks([2, 3, 4, 5,6,7,8,9])
plt.title('Silhouette scores vs Number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhoutte score')
#plt.savefig('Silhoutte_score.png')
plt.show()

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

clustering_spectral_3 = KMeans(n_clusters=2)
clustering_spectral_3.fit(PCA_components_s_3)

print('Silhoutte score of K-Mean clustering is ' + str(silhouette_score(PCA_components_s_3, clustering_spectral_3.labels_)))

In [ ]:
spectral_labels_3=clustering_spectral_3.labels_
spectral_labels_3

In [ ]:
spectral_final_3 = pd.concat([PCA_components_s_3, pd.DataFrame({'pca_clusters':spectral_labels_3})], axis=1)

In [ ]:
spectral_final_3.tail(3)

In [ ]:
spectral_final_3['pca_clusters'].value_counts()

In [ ]:
##### Clustering with ( Not All engineered Features)
#### When Cluster Number are 4
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(spectral_final_3.iloc[:,0],spectral_final_3.iloc[:,1], hue=spectral_final_3['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Distribution of cluster based on only Spectral Domain Features (PCA)',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

###### `********************************************************************************************************************`